In [1]:
import pandas as pd

## Easy

Построить прогноз по одной марке машин при помощи решающих деревьев, взять только числовые признаки. В качестве метрики взять `r2_score`. Оценить важность признаков, проинтерпретировать модель.


In [2]:
data = pd.read_csv("../data/merc.csv")

In [3]:
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0


In [4]:
from sklearn.model_selection import train_test_split

numeric_features = data.select_dtypes(include=['int64', 'float64'])
x_train, x_test, y_train, y_test = train_test_split(numeric_features.drop(columns=["price"]), data["price"])

In [5]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(x_train, y_train)

DecisionTreeRegressor()

In [6]:
from sklearn.metrics import r2_score

preds = model.predict(x_test)
print(r2_score(preds, y_test))
print(pd.DataFrame(zip(x_train.columns, model.feature_importances_)))

0.8472891768061602
            0         1
0        year  0.084790
1     mileage  0.333791
2         tax  0.033999
3         mpg  0.251445
4  engineSize  0.295975


Самым важным признаком оказался пробег, что, в целом понятно, т.к. пробег влияет на износ автомобиля, бла-бла-бла
Хотя, я тут глянул другие марки, там немного другие фичи решают больше, так что от производитя зависит, видимо!


## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

Я удалил из датасета файлы, начинающиеся с raw, т.к. они мешались

In [7]:
import os

hyundi_data = pd.read_csv("../data/hyundi.csv")
hyundi_data["tax"]=hyundi_data["tax(£)"]
hyundi_data.drop(columns=["tax(£)"], inplace=True)

def collect_files(directory: str) -> list:
    data_frames = []
    for file in os.listdir(directory):
        if file != "hyundi.csv":
            data_frames.append(pd.read_csv(os.path.join(directory, file)))
        
    return data_frames

dfs = collect_files("../data")
dfs.append(hyundi_data)
merged_data = pd.concat(dfs, ignore_index=True)

print(merged_data)

          model  year  price transmission  mileage fuelType    tax   mpg  \
0            A1  2017  12500       Manual    15735   Petrol  150.0  55.4   
1            A6  2016  16500    Automatic    36203   Diesel   20.0  64.2   
2            A1  2016  11000       Manual    29946   Petrol   30.0  55.4   
3            A4  2017  16800    Automatic    25952   Diesel  145.0  67.3   
4            A3  2019  17300       Manual     1998   Petrol  145.0  49.6   
...         ...   ...    ...          ...      ...      ...    ...   ...   
108535      I30  2016   8680       Manual    25906   Diesel    0.0  78.4   
108536      I40  2015   7830       Manual    59508   Diesel   30.0  65.7   
108537      I10  2017   6830       Manual    13810   Petrol   20.0  60.1   
108538   Tucson  2018  13994       Manual    23313   Petrol  145.0  44.8   
108539   Tucson  2016  15999    Automatic    11472   Diesel  125.0  57.6   

        engineSize  
0              1.4  
1              2.0  
2              1.4  
3  

In [13]:
categorial_featrues = ['model', 'transmission', 'fuelType']
dummy_df = pd.get_dummies(data=merged_data, columns=categorial_featrues, drop_first=True)
dummy_df = dummy_df.sample(int(len(dummy_df)*0.2), random_state=42)
dummy_df = dummy_df.dropna()

x_train, x_test, y_train, y_test = train_test_split(
    dummy_df.drop(columns=["price"]),
    dummy_df["price"],
    train_size=0.75
    )

In [14]:
dummy_df.head()

,year,price,mileage,tax,mpg,engineSize,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,...,model_200,model_220,model_230,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
64251,2017,7999,25000,150.0,57.7,1.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
3915,2017,18390,54421,30.0,61.4,2.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
11985,2019,24440,5639,145.0,49.6,2.0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
56419,2019,24700,12944,145.0,61.4,2.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
20299,2015,17399,70601,160.0,52.3,2.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer

params = {
    "criterion": ('squared_error', 'friedman_mse','absolute_error', 'poisson'),
    "max_depth": [3, 5, 7, 9, 11]
}

search = GridSearchCV(
    DecisionTreeRegressor(),
    params
)

search.fit(x_train, y_train)
search.best_estimator_

DecisionTreeRegressor(criterion='poisson', max_depth=11)

In [16]:
params, score = search.best_params_, search.best_score_
print(params, score)

{'criterion': 'poisson', 'max_depth': 11} 0.870220150580393


Получили, что лучшие параметры - вот над этой подписью, там и бест скор видно!